# Generate stores & customers for the demo

## Setup the environment

### Install packages

In [ ]:
!pip install faker
!pip install pymorton
!pip install v3io
!pip install v3io_frames
!pip install boto3

### imports

In [2]:
import logging
import os
import pickle
import random
import itertools
import boto3
import botocore

# Demo
import pandas as pd
import pymorton as pm
from faker import Faker
from faker.providers import BaseProvider

# DB
import v3io as v3c
import v3io_frames as v3f

### Define environment variables

In [3]:
# Stores
NUMBER_OF_STORES=50
STORES_TABLE='stores'
ACCURACY=20
BASE_ACCURACY=14

# Customers
NUMBER_OF_CUSTOMERS=5000
CUSTOMERS_TABLE='customers'
os.environ['CUSTOMERS_DATA_FILE_DIR'] = os.path.join(os.getcwd(), 'data')
os.environ['CUSTOMERS_DATA_FILE_PATH'] = os.path.join(os.environ['CUSTOMERS_DATA_FILE_DIR'], 'data_train_n.csv')
!mkdir -p ${CUSTOMERS_DATA_FILE_DIR}

In [4]:
!curl -L "iguazio-sample-data.s3.amazonaws.com/data_train_n.csv" > ${CUSTOMERS_DATA_FILE_PATH}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  399M  100  399M    0     0  16.8M      0  0:00:23  0:00:23 --:--:-- 17.5M


In [5]:
!ls -lah ${CUSTOMERS_DATA_FILE_DIR}

total 0
-rw-r--r-- 1 50 root 400M Jul 15 19:20 data_train_n.csv


### Open DB client

In [6]:
client = v3f.Client('framesd:8081')

## Generate Stores

### Location provider

In [7]:
class LocationProvider(BaseProvider):
    '''
    Creates locations within base_location

    Uses QuadTree for Geohashing
        @{http://tech.taskrabbit.com/blog/2015/06/09/elasticsearch-geohash-vs-geotree/}
        @{http://mapzen.github.io/leaflet-spatial-prefix-tree/}
        @{http://blog.notdot.net/2009/11/Damn-Cool-Algorithms-Spatial-indexing-with-Quadtrees-and-Hilbert-Curves}
    '''    
    def location(self, location_base: str, base_acc=10, acc=20):
        coordinates = location_base[:base_acc]
        for i in range(acc-len(coordinates)):
            coordinates += str(random.randint(0, 3))
        return coordinates

Add the location provider as a faker provider

In [8]:
faker = Faker()
faker.add_provider(LocationProvider)

### Define scenario environment (locations)

In [9]:
london_city = (51.514926, -0.089580)
london_city_south = (51.501593, -0.094942)
london_west = (51.512309, -0.128966)
london_south_west = (51.495022, -0.162268)

london_coordinates = []
london_coordinates.append(london_city)
london_coordinates.append(london_city_south)
london_coordinates.append(london_west)
london_coordinates.append(london_south_west)

london_coordinates_qt = list(map(lambda cooridnate: pm.interleave_latlng(*cooridnate), london_coordinates))
london_coordinates_qt

['03311311313011311011000321002320',
 '03311311311233323013031101320003',
 '03311311313010023000032330133111',
 '03311311311222300331010333220231']

### Define Store class

In [10]:
class Store(object):
    def __init__(self, store_id: int, name: str, location: str):
        self.store_id = store_id if store_id is not None else random.randint(1, int(os.genev('NUMBER_OF_STORES', 500)))
        self.location = location
        (self.lat, self.long) = pm.deinterleave_latlng(location)
        self.store_name = name
        self.customers = 0
    
    def json(self):
        json_store = {
            'id': self.store_id,
            'name': self.store_name,
            'location': self.location,
            'latitude': self.lat,
            'longitude': self.long,
            'customers': self.customers
        }
        return json_store

### Create stores

In [11]:
def create_stores(faker, number_of_stores: int=500):
    remaining = number_of_stores
    stores = []
    while remaining > 0:
        # Create stores
        stores_batch = [Store(faker.msisdn(), faker.company(), faker.location(random.choice(london_coordinates_qt), BASE_ACCURACY, ACCURACY)).json() 
              for i in range(remaining)]
        
        # Count for duplicate locations
        locations = list(map(lambda store: store['location'], stores_batch))
        locations = list(dict.fromkeys(locations))
        remaining -= len(locations)
        
        # Update stores list
        stores.append(stores_batch)

    stores = list(itertools.chain.from_iterable(stores))
    stores = pd.DataFrame.from_records(stores)
    stores = stores.set_index(['location'])
    stores = stores[~stores.index.duplicated(keep='first')]
    return stores

In [23]:
client.delete('kv', STORES_TABLE, if_missing=1)

In [24]:
stores = create_stores(faker, NUMBER_OF_STORES)
stores.head(5)

,customers,id,latitude,longitude,name
location,,,,,
03311311311222322031,0,0959706149157,51.501846,-0.163765,"Callahan, Brown and Mcintyre"
03311311311222333231,0,1597445538821,51.503220,-0.155525,Martin LLC
03311311313011223332,0,4011726542711,51.525536,-0.105057,Lopez LLC
03311311311222321033,0,7863348673467,51.499443,-0.161018,"Phillips, Howell and Perez"
03311311313011321022,0,4730622102572,51.521416,-0.096130,Jones LLC


### Upload to DB 

In [25]:
# client.delete('kv', STORES_TABLE)
client.write('kv', STORES_TABLE, stores)

In [26]:
client.read('kv', STORES_TABLE).head(5)

,customers,id,latitude,longitude,name
location,,,,,
03311311311222132122,0.0,9114953173203,51.491203,-0.157928,Ray Group
03311311313011232211,0.0,0246754570108,51.524506,-0.103340,Mitchell Ltd
03311311313010121123,0.0,8678727228820,51.510429,-0.116386,Peterson-Vazquez
03311311313010310032,0.0,1510138723395,51.515923,-0.114670,Castillo PLC
03311311311222003302,0.0,2312783322955,51.486397,-0.171661,Hall-Thomas


## Generate customers

In [27]:
class Customer(object):
    def __init__(self, id=0, location='0', products=""):
        self.id = id
        self.location = location
        self.products = products
        
    def json(self):
        json_customer = {
            'id': self.id,
            'location': self.location,
            'products': self.products
        }
        return json_customer

In [28]:
def create_customers(customers_data: pd.DataFrame, number_of_customers: int):
    available_customer = list(customers_data.user_id)
    customers = [Customer(id=i,
                          products=customers_data.loc[customers_data['user_id'] == random.choice(available_customer)]\
                                    .drop(['user_id', 'Unnamed: 0'], axis=1)\
                                    .to_json()).json() for i in range(number_of_customers)]
    customers = pd.DataFrame.from_records(customers)
    customers.set_index(['id'])
    return customers

In [34]:
# Load customers seed data
customers_data = pd.read_csv(os.environ['CUSTOMERS_DATA_FILE_PATH'])

# Create actual customers
customers = create_customers(customers_data, NUMBER_OF_CUSTOMERS)
customers.head(5)

,id,location,products
0,0,0,"{""user_product_reordered_ratio"":{""15957"":1.0,""..."
1,1,0,"{""user_product_reordered_ratio"":{""27934"":1.0,""..."
2,2,0,"{""user_product_reordered_ratio"":{""62837"":1.0,""..."
3,3,0,"{""user_product_reordered_ratio"":{""100426"":1.0,..."
4,4,0,"{""user_product_reordered_ratio"":{""35775"":1.0,""..."


## Upload to DB

In [35]:
client.delete('kv', CUSTOMERS_TABLE, if_missing=1)

In [ ]:
client.write('kv', CUSTOMERS_TABLE, dfs=customers)

In [41]:
print(f'Customers table sized: {client.read("kv", CUSTOMERS_TABLE).shape}')

Customers table sized: (4979, 3)


In [42]:
client.read('kv', CUSTOMERS_TABLE).head(5)

,id,location,products
index,,,
4715,4715.0,0,"{""user_product_reordered_ratio"":{""53470"":1.0,""..."
4282,4282.0,0,"{""user_product_reordered_ratio"":{""31818"":1.0,""..."
2134,2134.0,0,"{""user_product_reordered_ratio"":{""443975"":1.0,..."
781,781.0,0,"{""user_product_reordered_ratio"":{""1312"":1.0,""3..."
4017,4017.0,0,"{""user_product_reordered_ratio"":{""106204"":1.0,..."


## Generate predictions

In [48]:
client.delete('tsdb', 'predictions', if_missing=1)
#client.create('tsdb', 'predictions', attrs={'rate': '1/s'}, if_exists=1)

In [49]:
!tsdbctl create -c bigdata -t predictions -r 1/s